# Initiating connection

In [ ]:
from snowflake.snowpark import Session

# Automatically get the current Snowflake session (no credentials needed)
session = Session.builder.getOrCreate()
# Select your working database and schema
session.sql("USE DATABASE churn_modeling_db").collect()
session.sql("USE SCHEMA churn_modeling_schema").collect()

# Load data from the table created earlier
df = session.table("customer_data")

# Show a sample
df.show(10)


# Initiating Feature Store and registering Entity

In [ ]:
from snowflake.snowpark import Session
from snowflake.snowpark.functions import col
from snowflake.ml.feature_store import FeatureStore, FeatureView, Entity, CreationMode

# Assume session is already created
# session = Session.builder.configs(...).create()

# Initialize the Feature Store
fs = FeatureStore(
    session=session,
    database="CHURN_MODELING_DB",
    name="CHURN_FEATURE_STORE",
    default_warehouse="CHURN_WAREHOUSE",
    creation_mode=CreationMode.CREATE_IF_NOT_EXIST
)

# Define an entity
customer_entity = Entity(
    name="Customer_v2",
    join_keys=["CLIENTID"],  # Make sure this matches your table key column
    desc="Customer entity for churn"
)
fs.register_entity(customer_entity)


# Prepping data to Feature View

In [ ]:
CREATE SCHEMA IF NOT EXISTS CHURN_MODELING_DB.CHURN_FEATURE_STORE;

In [ ]:
df_features = (
    df.with_column("CLIENTID", df["CLIENTID"])  # use existing CLIENTID
      .with_column("owns_credit_card", df["OWNSCREDITCARD"].cast("int"))
      .with_column("is_active", df["ISACTIVE"].cast("int"))
      .with_column("gender_enc", (df["GENDER"] == "Male").cast("int"))
      .with_column(
          "geo_enc",
          (df["GEOGRAPHY"] == "France").cast("int") * 0 +
          (df["GEOGRAPHY"] == "Spain").cast("int") * 1
      )
      .select(
          "CLIENTID",
          "CREDITSCORE",
          "AGE",
          "GRADE",
          "ACCOUNTBALANCE",
          "PRODUCTCOUNT",
          "owns_credit_card",
          "is_active",
          "SALARYESTIMATED",
          "gender_enc",
          "geo_enc",
          "CHURNED"
      )
)


# Grant accesses

In [ ]:
from snowflake.snowpark import Session

# Assume session is already created
# session = Session.builder.configs(...).create()

# 1️⃣ Get current role dynamically
current_role = session.sql("SELECT CURRENT_ROLE()").collect()[0][0]
print(f"Current role: {current_role}")

# 2️⃣ Build and execute grant statements dynamically
database_name = "CHURN_MODELING_DB"
schema_name = "churn_modeling_schema"
table_name = "CUSTOMER_DATA"

grant_statements = [
    f"GRANT USAGE ON DATABASE {database_name} TO ROLE {current_role}",
    f"GRANT USAGE ON SCHEMA {database_name}.{schema_name} TO ROLE {current_role}",
    f"GRANT SELECT ON TABLE {database_name}.{schema_name}.{table_name} TO ROLE {current_role}"
]

# Execute each statement
for stmt in grant_statements:
    print(f"Executing: {stmt}")
    session.sql(stmt).collect()


# Creating Feature View


In [ ]:
from snowflake.snowpark import Session
from snowflake.snowpark.functions import col
from snowflake.ml.feature_store import FeatureStore, FeatureView, Entity, CreationMode

# Create persistent view in the feature store schema
df_features.create_or_replace_view("churn_modeling_schema.CUSTOMER_FEATURES_VIEW")

# Create FeatureView
fv = FeatureView(
    name="customer_churn_features_v2",
    entities=[customer_entity],
    feature_df=session.table("churn_modeling_schema.CUSTOMER_FEATURES_VIEW"),
    timestamp_col=None,
    refresh_freq=None,
    desc="Features for customer churn prediction"
)

# Register FeatureView
registered_fv = fs.register_feature_view(
    feature_view=fv,
    version="v1",
    block=True
)

print("Feature View registered:", registered_fv)
